In [8]:
# Загрузка данных
with open('/content/Viking.txt', 'r', encoding='utf-8') as file:
    script_lines = file.read()

In [2]:
script_lines

'"x"\n"\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t(adsbygoogle = window.adsbygoogle || []).push({});\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t(Rain patters)\n(Loud thunderclap)\n(Heartbeat pulses loudly)\n(Men scream in agony)\n(Inhales deeply)\nLatvian warrior: (Attack yell)\n(Fighting grunts)\nAgghhh!\nUngh!\nOhhh!\nAgghhh!\nAgh! Ugh!\nAgghhhh!\n(Latvian warrior yells)\nScandinavian warrior: Ragnar!\nAgggghhhh! (Weapons clank)\nUngh! Ungh!\n(Hunting knife slices) Agh!\nUngh! Ungh!\n(Gurgling grunt)\n(Hard blow)\n(Panting)\n(Panting)\nUngh!\nAgghhh!\n(Breathing hard)\n(Breathing hard)\n(Exhausted exhales)\n(Exhausted exhale)\n(Thunder rumbles, rain patters)\n(Ravens caw)\n(Flesh rips)\n(Breathing hard)\n(Ravens caw loudly)\n(Ravens caw loudly)\n(Wings beat loudly)\nVoice: Odin!\n(Ominous howling)\n(Ravens caw loudly)\n(Heartbeat pulses)\n(Ravens caw loudly)\n(Thunder rumbles)\nRollo: Ragnar!\n<U+266A> More, give me more <U+266A>\n<U+266A> give me more\n<U+266A

In [16]:
import re

def extract_ragnar_dialogues(text):
    ragnar_dialogues = []
    lines = text.split("\n")
    for line in lines:
        if "Ragnar:" in line:
            dialogue = re.sub(r'\([^)]*\)', '', line)  # Удаляем текст в скобках
            dialogue = re.sub(r'\<[^>]*\>', '', dialogue)  # Удаляем текст в угловых скобках
            dialogue = re.sub(r'Ragnar:', '', dialogue)
            ragnar_dialogues.append(dialogue.strip())
        elif re.search(r'\bRagnar\b(?!\:)', line):
            continue
        # elif "Ragnar" in line and ":" not in line:
        #     dialogue = re.sub(r'\([^)]*\)', '', line)
        #     dialogue = re.sub(r'\<[^>]*\>', '', dialogue)
        #     ragnar_dialogues.append(dialogue.strip())
    return ragnar_dialogues

In [17]:
ragnar_dialogues = extract_ragnar_dialogues(script_lines)
for dialogue in ragnar_dialogues:
    print(dialogue)


The Earl will deal with some criminal offences and then, we'll discuss the summer raids.
The Earl decides.
I went to confess my love to her but I was set upon by a bear and an enormous hound who guarded her home.
And...?
That's right.
Say I wasn't there.
Why don't you go lie down, hmm?
I know.
Now, this candle is the sun.
That's it!
He wants to die well, without fear, to atone for his sins.
He should not have done that.
My lord, we all want to feast, but we also want to know where we will be raiding this summer.
Every year we go to the same places!
Let me see!
Stay with your Uncle.
I can't be sure.
To talk to the gods.
I want to know what the gods have in store.
To have this great future, must I challenge the law?
Wait outside.
We have someone special to visit.
Yes, only different.
Floki, this is my son, Bjorn.
Why unfortunately?
Floki is a boat-builder...
So... what about our boat?
The hull is deeper. How will my men set their oars?
For the anchor.
Hmm.
Let him stay awhile.
Hey!
We a

In [19]:
def save_dialogues_to_txt(dialogues, filename):
    with open(filename, "w", encoding="utf-8") as file:
        for dialogue in dialogues:
            file.write(dialogue + "\n")

# Применяем функцию для сохранения диалогов в файл
save_dialogues_to_txt(ragnar_dialogues, "ragnar_dialogues.txt")

In [21]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Загрузка диалогов персонажа Рагнара
dialogues = open("/content/ragnar_dialogues.txt").read().split("\n")

In [23]:
# Предварительная обработка данных
tokenizer = Tokenizer()
tokenizer.fit_on_texts(dialogues)
total_words = len(tokenizer.word_index) + 1

In [24]:
# Создание последовательностей
input_sequences = []
for line in dialogues:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [25]:
# Паддинг последовательностей
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [26]:
# Создание обучающих данных
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [27]:
# Создание модели нейронной сети
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(total_words, 100, input_length=max_sequence_len-1),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(150)),
    tf.keras.layers.Dense(total_words, activation='softmax')
])

In [28]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
history = model.fit(xs, ys, epochs=100, verbose=1)

Epoch 1/100
49/49 [==============================] - 13s 195ms/step - loss: 6.0189 - accuracy: 0.0413
Epoch 2/100
49/49 [==============================] - 9s 190ms/step - loss: 5.6452 - accuracy: 0.0432
Epoch 3/100
49/49 [==============================] - 8s 162ms/step - loss: 5.5856 - accuracy: 0.0458
Epoch 4/100
49/49 [==============================] - 9s 182ms/step - loss: 5.5038 - accuracy: 0.0510
Epoch 5/100
49/49 [==============================] - 9s 182ms/step - loss: 5.3591 - accuracy: 0.0574
Epoch 6/100
49/49 [==============================] - 8s 157ms/step - loss: 5.1595 - accuracy: 0.0652
Epoch 7/100
49/49 [==============================] - 9s 183ms/step - loss: 4.9294 - accuracy: 0.0929
Epoch 8/100
49/49 [==============================] - 9s 183ms/step - loss: 4.6854 - accuracy: 0.1226
Epoch 9/100
49/49 [==============================] - 8s 157ms/step - loss: 4.4174 - accuracy: 0.1445
Epoch 10/100
49/49 [==============================] - 9s 184ms/step - loss: 4.1545 - accur

In [30]:
# Функция для генерации ответа
def generate_response(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [32]:
# Пример использования
seed_text = "What will happen at the thing?"
next_words = 10
response = generate_response(seed_text, next_words, model, max_sequence_len)
print(response)

1/1 [==============================] - 0s 67ms/step
What will happen at the thing? uncle in your claim but for you lead his army


In [33]:
# Сохранение модели
model.save("ragnar_model")

# Сохранение токенизатора
import pickle

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)